In [3]:
# project imports
import sys
sys.path.append('/home/mark/Documents/code/drone/sumo/utils')
from runSumo import runSumo
from sumo_loop import sumo_loop

# general imports
import os
import subprocess
import numpy as np
import scipy.linalg
from mpl_toolkits.mplot3d import Axes3D
import pickle
import pandas as pd

# plotting params
from IPython.display import Math
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams["axes.spines.right"] = False
mpl.rcParams["axes.spines.top"] = False
mpl.rcParams["lines.linewidth"] = 2

In [2]:
in_dir = '/home/mark/Documents/code/drone/sumo/results/capacities/2_all.p'
r_vs_f = pickle.load(open(in_dir,'rb'))
capacity = np.max(r_vs_f, axis=1)*3600
ratios = np.logspace(np.log10(0.001),np.log10(0.5),100)
flows = np.arange(10,5000,10) 



In [3]:
data = []

for r_idx in range(len(capacity)):
    for f_idx in range(len(flows)):
        x = flows[f_idx]*ratios[r_idx]
        y = flows[f_idx]
        z = capacity[r_idx]
        data.append([x,y,z])

data = np.asarray(data)

In [4]:
# regular grid covering the domain of the data
X,Y = np.meshgrid(np.arange(10,2510,10), flows)
XX = X.flatten()
YY = Y.flatten()


# best-fit linear plane
A_lin = np.c_[data[:,0], data[:,1], np.ones(data.shape[0])]
C_lin,_,_,_ = scipy.linalg.lstsq(A_lin, data[:,2])    # coefficients

# evaluate it on grid
Z_lin = C_lin[0]*X + C_lin[1]*Y + C_lin[2]

# or expressed using matrix/vector product
#Z = np.dot(np.c_[XX, YY, np.ones(XX.shape)], C).reshape(X.shape)


# best-fit quadratic curve
A_q = np.c_[np.ones(data.shape[0]), data[:,:2], np.prod(data[:,:2], axis=1), data[:,:2]**2]
C_q,_,_,_ = scipy.linalg.lstsq(A_q, data[:,2])

# evaluate it on a grid
Z_q = np.dot(np.c_[np.ones(XX.shape), XX, YY, XX*YY, XX**2, YY**2], C_q).reshape(X.shape)

In [5]:
%matplotlib qt
# plot points and fitted surface
fig = plt.figure(figsize=(12,4))
ax2 = fig.add_subplot(1,2,1,projection='3d')
ax3 = fig.add_subplot(1,2,2,projection='3d')


ax2.set_xlim3d(0,2500)
ax2.set_ylim3d(0,5000)
ax2.set_zlim3d(1000,3000)
ax3.set_xlim3d(0,2500)
ax3.set_ylim3d(0,5000)
ax3.set_zlim3d(0,2500)


# ax2.view_init(elev=5., azim=-90)
# ax3.view_init(elev=5., azim=-90)
# first we create the mesh

# next we do the linear fit

ax2.set_title('Best Linear Fit')
ax2.plot_surface(X, Y, Z_lin, rstride=1, cstride=1, alpha=0.2)
ax2.scatter(data[:,0], data[:,1], data[:,2], c='r', s=0.1)

# finally we do quadratic fit
ax3.set_title('Best Quadratic Fit')
ax3.plot_surface(X, Y, Z_q, rstride=1, cstride=1, alpha=0.2)
ax3.scatter(data[:,0], data[:,1], data[:,2], c='r', s=0.1)

# 

# plt.xlabel('X')
# plt.ylabel('Y')
# ax.set_zlabel('Z')
# ax.axis('auto')
# ax.axis('tight')
plt.show()

In [6]:
print(C_lin)

[-1.23716255e+00  1.01475586e-01  3.56492000e+03]


In [7]:
print(C_q)

[ 3.56492000e+03 -3.29909462e+00  2.70601110e-01  4.06261516e-04
  3.75957445e-04 -4.12888233e-05]
